# Talk to your App

In majority of the examples, LLM Usage is shown as a chat-bot or personal assistant. 

A more powerful usage of Agentic-AI are when they interact with the system in the background and enrich it or take autonomous actions. This is conceptually similar to streaming analytics - with analytics being processed by Agents powered by LLMs.

You will see that for chat-bot type usage - if the AI infrastructure is down, things still work.
However for the Agents in backend - if the AI system is down, then the system will suffer some down time.

_Each module is typically dependent on the prior modules having been completed successfully_

In [2]:
import openai
import re
import httpx
import os
import rich
import json
from openai import OpenAI
from agents import Agent, ModelSettings, function_tool,Runner,AsyncOpenAI,OpenAIChatCompletionsModel
# from rich.pretty import pprint

api_key = "placeholder" 
# model="mistral-small:latest"

model = "qwen3:32b"
base_url = "http://localhost:11434/v1/"

model = OpenAIChatCompletionsModel( 
    model=model,
    openai_client=AsyncOpenAI(base_url=base_url, api_key=api_key)
)

print("Setup complete")

Setup complete


## Automated Error Handling

1.The process is started when an ansible job log completion data comes in. (as of now, we simulate this as a user input)
1. It examimes if there is any error. If no error, it ends
1. If there is an error:
    - Agent analyzes and recommends
    - Agent opens a jira ticket
    - Agent sends a slack message

![Workflow](resources/images/agent_log_workflow.png)

In [3]:
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace


@dataclass
class Advisor:
    body: str

@dataclass
class Slacker:
    body: str

@dataclass
class JIRAer:
    body: str


advisor_agent = Agent(
    name="advisor_agent",
    instructions="You can look at the contents of an ansible log and spot the error. You will describe what the error is in a few crisp sentences so that a human can take corrective actions.",
    model = model,
    output_type=Advisor,
)

slack_agent = Agent(
    name="slack_agent",
    instructions="If there is an error captured in the input, you will always state - I have slacked the message. Also add the contents that in the provided input. Give a made up slack link. If there is no error, then just say - All is well, there is nothing to be done.",
    model = model,
    output_type=Slacker,
)

jira_agent = Agent(
    name="jira_agent",
    instructions="If there is an error captured in the input, you will always state - I have opened a JIRA ticket. Also add the contents that in the provided input. Give a made up JIRA Handle. If there is no error, then just say - All is well, there is nothing to be done.",
    model = model,
    output_type=JIRAer,
)



In [4]:
#msg = "all tasks successfully finished"
msg = "could not connect to the host as the password as expired. "
inputs = [{"content": msg, "role": "user"}]

#with trace("Router"):
#    story_outline_result = await Runner.run(advisor_agent,inputs)
#    #uncomment this to see the details
#    pprint(story_outline_result)
#    print("--------------------------")
#    print(story_outline_result.final_output)

with trace("Workflow"):
    print("----------Advisor Output----------")
    advisor_result = await Runner.run(advisor_agent,inputs)
    print(advisor_result.final_output.body)
    advisor_output: Advisor = advisor_result.final_output
    
    print("----------JIRA Output----------")
    jira_result = await Runner.run(jira_agent,advisor_output.body)
    print(jira_result.final_output.body)
    
    print("----------Slack Output----------")
    slack_result = await Runner.run(slack_agent,advisor_output.body)
    print(slack_result.final_output.body)

----------Advisor Output----------
The error message indicates that the connection to the host failed because the user's password has expired. You need to update the password for the user on the target host to resolve this issue. You can either set a new password directly on the host using the 'passwd' command or configure the user to not have their password expire using the 'chage' command. Once the password is updated, try running the Ansible playbook again.
----------JIRA Output----------
I have opened a JIRA ticket. Also add the contents that in the provided input. Give a made up JIRA Handle. If there is no error, then just say - All is well, there is nothing to be done.
----------Slack Output----------
I have slacked the message. Please refer to the link for more details: https://slack.com/messages/expired-password-error/123456789


# Human-in-the-Loop
But LLMs hallucinate!

Yes, LLMs, just like traditional AI and human beings may not always give the right answer. To handle those kind of possible mistakes, we have processes in place.

In the above example, we are seamlessly blending human-in-the-loop when we open a JIRA ticket or Slack a message. Even if the contents of these are not entirely accurate, we do not lose much because a person can check and correct if needed. 

In distributed systems, there are lots of similar examples - which has been around us for a long time - to take care of possible errors: for example those which arise out of CAP Theorem related inconsistencies. 
